# Convert yolo.pt to yolo.engine

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

# Export dengan optimasi maksimal untuk speed
model.export(
    format='engine',
    device=0,
    half=True,           # FP16 precision untuk speed
    batch=32,             # Batch size untuk throughput
    workspace=4,         # Workspace memory (GB)
    imgsz=640,          # Input size
    verbose=False
)


# Test Model Tensorflow yolo.engine

In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
import threading
import queue
from collections import deque
import torch

class BatchedSharedMemorySystem:
    def __init__(self, engine_path, video_path, batch_size=32, buffer_size=50):
        self.model = YOLO(engine_path)
        self.video_path = video_path
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        
        # Shared Memory Buffers sesuai flowchart
        self.frame_buffer = queue.Queue(maxsize=buffer_size)
        self.detection_buffer = queue.Queue(maxsize=buffer_size)
        self.tracking_buffer = queue.Queue(maxsize=buffer_size)
        
        # Thread control
        self.running = True
        
        # Tracking state
        self.tracks = {}
        self.next_track_id = 0
        
        # Performance monitoring
        self.stats = {
            'camera_fps': 0,
            'detection_fps': 0,
            'tracking_fps': 0,
            'batch_fps': 0,
            'camera_frames': 0,
            'detection_frames': 0,
            'tracking_frames': 0,
            'batches_processed': 0,
            'start_time': time.time()
        }
        
        # Frame dimensions
        self.frame_height = 480
        self.frame_width = 640
        self.frame_channels = 3
    
    def camera_thread(self):
        """Camera Thread - Continuous frame capture ke Frame Buffer"""
        cap = cv2.VideoCapture(self.video_path)
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        cap.set(cv2.CAP_PROP_FPS, 120)
        
        frame_count = 0
        fps_start = time.time()
        
        print("📹 Camera Thread Started")
        
        while self.running:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            # Resize frame
            frame = cv2.resize(frame, (self.frame_width, self.frame_height))
            
            # Add frame to Frame Buffer
            if not self.frame_buffer.full():
                self.frame_buffer.put((frame.copy(), frame_count, time.time()))
                frame_count += 1
                self.stats['camera_frames'] += 1
            
            # Calculate camera FPS
            if frame_count % 100 == 0:
                elapsed = time.time() - fps_start
                self.stats['camera_fps'] = 100 / elapsed if elapsed > 0 else 0
                fps_start = time.time()
                print(f"📹 Camera FPS: {self.stats['camera_fps']:.1f}")
        
        cap.release()
        print("📹 Camera Thread Stopped")
    
    def object_detection_thread(self):
        """Object Detection Thread - Batch processing 32 frames"""
        detection_count = 0
        fps_start = time.time()
        
        print(f"🔍 Object Detection Thread Started (Batch Size: {self.batch_size})")
        
        while self.running:
            try:
                frames_batch = []
                frame_ids = []
                timestamps = []
                
                # Collect batch_size frames dari Frame Buffer
                for _ in range(self.batch_size):
                    try:
                        frame, frame_id, timestamp = self.frame_buffer.get(timeout=0.1)
                        frames_batch.append(frame)
                        frame_ids.append(frame_id)
                        timestamps.append(timestamp)
                    except queue.Empty:
                        break
                
                if len(frames_batch) > 0:
                    while len(frames_batch) < self.batch_size:
                        frames_batch.append(frames_batch[-1])
                        frame_ids.append(frame_ids[-1])
                        timestamps.append(timestamps[-1])
                    
                    with torch.no_grad():
                        batch_results = self.model.predict(
                            frames_batch,
                            device=0,
                            conf=0.25,
                            iou=0.45,
                            verbose=False,
                            stream=False
                        )
                    
                    # Put batch results ke Detection Buffer
                    for i, (frame, result, frame_id, timestamp) in enumerate(zip(frames_batch, batch_results, frame_ids, timestamps)):
                        if not self.detection_buffer.full():
                            self.detection_buffer.put((frame, result, frame_id, timestamp))
                            detection_count += 1
                            self.stats['detection_frames'] += 1
                    
                    self.stats['batches_processed'] += 1
                    
                    # Calculate detection FPS
                    if detection_count % 100 == 0:
                        elapsed = time.time() - fps_start
                        self.stats['detection_fps'] = 100 / elapsed if elapsed > 0 else 0
                        self.stats['batch_fps'] = (self.stats['batches_processed'] * self.batch_size) / (time.time() - self.stats['start_time'])
                        fps_start = time.time()
                        print(f"🔍 Detection FPS: {self.stats['detection_fps']:.1f} | Batch FPS: {self.stats['batch_fps']:.1f}")
                
            except Exception as e:
                print(f"Detection error: {e}")
        
        print("🔍 Object Detection Thread Stopped")
    
    def object_tracking_thread(self):
        """Object Tracking Thread - Individual frame tracking"""
        tracking_count = 0
        fps_start = time.time()
        
        print("🎯 Object Tracking Thread Started")
        
        while self.running:
            try:
                # Get detection result
                frame, detection_result, frame_id, timestamp = self.detection_buffer.get(timeout=0.1)
                
                # Object Tracking per frame
                tracked_frame = self.perform_tracking(frame, detection_result, frame_id)
                
                # Put to Tracking Buffer
                if not self.tracking_buffer.full():
                    self.tracking_buffer.put((tracked_frame, frame_id, timestamp))
                    tracking_count += 1
                    self.stats['tracking_frames'] += 1
                
                # Calculate tracking FPS
                if tracking_count % 100 == 0:
                    elapsed = time.time() - fps_start
                    self.stats['tracking_fps'] = 100 / elapsed if elapsed > 0 else 0
                    fps_start = time.time()
                    print(f"🎯 Tracking FPS: {self.stats['tracking_fps']:.1f}")
                
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Tracking error: {e}")
        
        print("🎯 Object Tracking Thread Stopped")
    
    def perform_tracking(self, frame, detection_result, frame_id):
        """Perform object tracking dengan ID assignment"""
        tracked_frame = frame.copy()
        
        if detection_result.boxes is not None:
            boxes = detection_result.boxes.xyxy.cpu().numpy()
            confs = detection_result.boxes.conf.cpu().numpy()
            classes = detection_result.boxes.cls.cpu().numpy()
            
            current_detections = []
            
            # Extract detections
            for box, conf, cls in zip(boxes, confs, classes):
                if conf > 0.25:
                    x1, y1, x2, y2 = map(int, box)
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    
                    current_detections.append({
                        'bbox': (x1, y1, x2, y2),
                        'center': (center_x, center_y),
                        'conf': conf,
                        'class': int(cls)
                    })
            
            # Assign track IDs
            for detection in current_detections:
                track_id = self.assign_track_id(detection)
                
                x1, y1, x2, y2 = detection['bbox']
                
                # Draw tracking visualization
                color = self.get_track_color(track_id)
                cv2.rectangle(tracked_frame, (x1, y1), (x2, y2), color, 2)
                
                # Draw track info
                label = f"ID:{track_id} C:{detection['class']} {detection['conf']:.2f}"
                cv2.putText(tracked_frame, label, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
                # Draw center point
                cv2.circle(tracked_frame, detection['center'], 3, color, -1)
        
        return tracked_frame
    
    def assign_track_id(self, detection):
        """Assign track ID menggunakan distance-based matching"""
        center = detection['center']
        min_distance = float('inf')
        assigned_id = None
        
        # Find closest existing track
        for track_id, track_data in list(self.tracks.items()):
            if time.time() - track_data['last_seen'] > 2.0:
                del self.tracks[track_id]
                continue
                
            track_center = track_data['center']
            distance = np.sqrt((center[0] - track_center[0])**2 + (center[1] - track_center[1])**2)
            
            if distance < min_distance and distance < 80:
                min_distance = distance
                assigned_id = track_id
        
        # Create new track if no match
        if assigned_id is None:
            assigned_id = self.next_track_id
            self.next_track_id += 1
        
        # Update track
        self.tracks[assigned_id] = {
            'center': center,
            'last_seen': time.time(),
            'class': detection['class']
        }
        
        return assigned_id
    
    def get_track_color(self, track_id):
        """Generate consistent color untuk setiap track ID"""
        colors = [
            (0, 255, 0),    # Green
            (255, 0, 0),    # Blue
            (0, 0, 255),    # Red
            (255, 255, 0),  # Cyan
            (255, 0, 255),  # Magenta
            (0, 255, 255),  # Yellow
            (128, 0, 128),  # Purple
            (255, 165, 0),  # Orange
        ]
        return colors[track_id % len(colors)]
    
    def display_thread(self):
        """Display thread untuk visualization"""
        print("🖥️ Display Thread Started")
        
        while self.running:
            try:
                # Get tracked frame from Tracking Buffer
                tracked_frame, frame_id, timestamp = self.tracking_buffer.get(timeout=0.1)
                
                # Add performance overlay
                self.add_performance_overlay(tracked_frame)
                
                # Display frame
                cv2.imshow('Batched Multi-Thread Object Detection & Tracking', tracked_frame)
                
                # Handle keyboard input
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    self.running = False
                    break
                elif key == ord('r'):
                    self.tracks.clear()
                    self.next_track_id = 0
                    print("🔄 Tracking reset")
                
            except queue.Empty:
                continue
        
        cv2.destroyAllWindows()
        print("🖥️ Display Thread Stopped")
    
    def add_performance_overlay(self, frame):
        """Add performance statistics overlay"""
        elapsed = time.time() - self.stats['start_time']
        
        # Calculate overall FPS (bottleneck from all thread)
        overall_fps = min(self.stats['camera_fps'], self.stats['detection_fps'], self.stats['tracking_fps'])
        
        stats_text = [
            f"Camera FPS: {self.stats['camera_fps']:.1f}",
            f"Detection FPS: {self.stats['detection_fps']:.1f}",
            f"Batch FPS: {self.stats['batch_fps']:.1f}",
            f"Tracking FPS: {self.stats['tracking_fps']:.1f}",
            f"Overall FPS: {overall_fps:.1f}",
            f"Batch Size: {self.batch_size}",
            f"Batches Processed: {self.stats['batches_processed']}",
            f"Active Tracks: {len(self.tracks)}",
            f"Buffer: F{self.frame_buffer.qsize()}/D{self.detection_buffer.qsize()}/T{self.tracking_buffer.qsize()}",
            f"Runtime: {elapsed:.1f}s"
        ]
        
        # Background for text
        overlay = frame.copy()
        cv2.rectangle(overlay, (5, 5), (400, 220), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        
        # Add text
        for i, text in enumerate(stats_text):
            y_pos = 20 + (i * 20)
            color = (0, 255, 0) if "FPS" in text and float(text.split(":")[1].strip()) > 100 else (0, 255, 255)
            cv2.putText(frame, text, (10, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # Success indicator
        if self.stats['batch_fps'] > 200:
            cv2.putText(frame, "🎉 TARGET 200+ BATCH FPS ACHIEVED!", (10, 250), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    def run(self):
        print("🚀 Starting Batched Multi-Thread Object Detection & Tracking System")
        print(f"Architecture: Camera → Frame Buffer → Batch Detection (32) → Detection Buffer → Tracking → Tracking Buffer → Display")
        print(f"Batch Size: {self.batch_size}")
        print("Press 'q' to quit, 'r' to reset tracking")
        
        # Start all threads
        threads = [
            threading.Thread(target=self.camera_thread, daemon=True, name="CameraThread"),
            threading.Thread(target=self.object_detection_thread, daemon=True, name="BatchDetectionThread"),
            threading.Thread(target=self.object_tracking_thread, daemon=True, name="TrackingThread"),
            threading.Thread(target=self.display_thread, daemon=True, name="DisplayThread")
        ]
        
        # Start threads
        for thread in threads:
            thread.start()
            print(f"✅ {thread.name} started")
        
        try:
            # Monitor system performance
            while self.running:
                time.sleep(1)
                
                # Check target achievement
                if self.stats['batch_fps'] > 200:
                    print(f"🎉 BATCH TARGET ACHIEVED: {self.stats['batch_fps']:.1f} FPS!")
                
        except KeyboardInterrupt:
            print("\n⏹️ Stopping system...")
            self.running = False
        
        # Wait for threads to finish
        for thread in threads:
            if thread.is_alive():
                thread.join(timeout=2.0)
                print(f"✅ {thread.name} stopped")
        
        # Final statistics
        elapsed = time.time() - self.stats['start_time']
        print(f"\n📊 Final Statistics:")
        print(f"Runtime: {elapsed:.2f}s")
        print(f"Camera frames: {self.stats['camera_frames']}")
        print(f"Detection frames: {self.stats['detection_frames']}")
        print(f"Tracking frames: {self.stats['tracking_frames']}")
        print(f"Batches processed: {self.stats['batches_processed']}")
        print(f"Final Camera FPS: {self.stats['camera_fps']:.2f}")
        print(f"Final Detection FPS: {self.stats['detection_fps']:.2f}")
        print(f"Final Batch FPS: {self.stats['batch_fps']:.2f}")
        print(f"Final Tracking FPS: {self.stats['tracking_fps']:.2f}")

# Usage
if __name__ == "__main__":
    engine_path = "yolo11m.engine"  # use model TensorRT
    # engine_path = "yolo11m.pt"
    video_path = "C:/Users/JuhenFW/Downloads/4K Road traffic video for object detection and tracking - free download now.mp4/4K Road traffic video for object detection and tracking - free download now.mp4"
    
    # Batch size 32
    batch_size = 32
    buffer_size = 50
    
    system = BatchedSharedMemorySystem(engine_path, video_path, batch_size, buffer_size)
    system.run()
